Libraries

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

PROJECT_ROOT = Path("..").resolve()

DATA_DIR = PROJECT_ROOT / "data" / "Converted Images"

ANNOT_DIR = PROJECT_ROOT / "data" / "Annotated Files"

OUT_DIR = PROJECT_ROOT / "outputs"
SAMPLE_DIR = OUT_DIR / "samples" / "segmentation_guided"
SAMPLE_DIR.mkdir(parents=True, exist_ok=True)

EXTS = {".jpg", ".jpeg", ".png", ".bmp"}
RANDOM_STATE = 42


Scan Dataset & Binary Labeling

In [2]:
def scan_dataset(data_dir: Path):
    class_dirs = sorted([p for p in data_dir.iterdir() if p.is_dir()])
    rows = []
    for cls_dir in class_dirs:
        cls = cls_dir.name
        y = 0 if cls.lower() == "healthy" else 1
        for fp in cls_dir.rglob("*"):
            if fp.is_file() and fp.suffix.lower() in EXTS:
                rows.append({"path": str(fp), "class": cls, "Output": y})
    return pd.DataFrame(rows)

df = scan_dataset(DATA_DIR)

print("Counts per folder:")
display(df["class"].value_counts())

print("\nBinary counts (Output):")
display(df["Output"].value_counts())

print("\nTotal:", len(df))


Counts per folder:


class
Healthy            136
Soft_Rot           129
Gray_Blight        119
Brown_Stem_Spot    119
Anthracnose        118
Stem_Canker        103
Name: count, dtype: int64


Binary counts (Output):


Output
1    588
0    136
Name: count, dtype: int64


Total: 724


K-Means Segmentation

In [3]:
def kmeans_labels_lab(bgr, k=3):
    lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)
    H, W = lab.shape[:2]
    X = lab.reshape(-1, 3).astype(np.float32)

    km = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init="auto")
    labels = km.fit_predict(X).reshape(H, W)
    return labels

def build_annot_index(annot_root: Path):
    idx = {}
    if not annot_root.exists():
        return idx
    for p in annot_root.rglob("*.txt"):
        idx[p.stem] = p
    return idx

ANNOT_INDEX = build_annot_index(ANNOT_DIR)
print("Annotation txt files found:", len(ANNOT_INDEX))

def find_annotation_txt(img_path: str):
    stem = Path(img_path).stem
    return ANNOT_INDEX.get(stem, None)

def parse_yolo_txt(txt_path: Path):
    boxes = []
    with open(txt_path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            if len(parts) != 5:
                continue
            cls = int(float(parts[0]))
            xc, yc, bw, bh = map(float, parts[1:])
            boxes.append((cls, xc, yc, bw, bh))
    return boxes

def yolo_to_xyxy(box, W, H):
    _, xc, yc, bw, bh = box
    x1 = (xc - bw/2) * W
    y1 = (yc - bh/2) * H
    x2 = (xc + bw/2) * W
    y2 = (yc + bh/2) * H
    x1 = int(max(0, round(x1))); y1 = int(max(0, round(y1)))
    x2 = int(min(W-1, round(x2))); y2 = int(min(H-1, round(y2)))
    return x1, y1, x2, y2

def lesion_mask_from_txt(shape_hw, txt_path: Path):
    H, W = shape_hw
    mask = np.zeros((H, W), dtype=np.uint8)
    boxes = parse_yolo_txt(txt_path)
    for b in boxes:
        x1, y1, x2, y2 = yolo_to_xyxy(b, W, H)
        mask[y1:y2+1, x1:x2+1] = 1
    return mask

def pick_cluster_by_overlap(labels, lesion_mask):
    lesion_area = int(lesion_mask.sum())
    if lesion_area == 0:
        return None, None

    k = int(labels.max()) + 1
    best_k, best_score = None, -1.0
    for kk in range(k):
        cluster_mask = (labels == kk).astype(np.uint8)
        inter = int((cluster_mask & lesion_mask).sum())
        score = inter / (lesion_area + 1e-9)  
        if score > best_score:
            best_score = score
            best_k = kk
    return best_k, float(best_score)

def refine_roi_mask(mask01):
    mask = (mask01 > 0).astype(np.uint8)
    kernel = np.ones((7,7), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN,  kernel, iterations=1)

    n, cc, stats, _ = cv2.connectedComponentsWithStats(mask*255, connectivity=8)
    if n <= 1:
        return mask
    areas = stats[1:, cv2.CC_STAT_AREA]
    idx = 1 + int(np.argmax(areas))
    return (cc == idx).astype(np.uint8)

def segment_guided(bgr, txt_path: Path):
    labels = kmeans_labels_lab(bgr, k=3)
    lesion_mask = lesion_mask_from_txt(labels.shape, txt_path)

    roi_k, score = pick_cluster_by_overlap(labels, lesion_mask)
    if roi_k is None:
        return labels, lesion_mask, None, None

    roi_mask = (labels == roi_k).astype(np.uint8)
    roi_mask = refine_roi_mask(roi_mask)
    return labels, lesion_mask, roi_mask, score


Annotation txt files found: 545


Visualization & Save Samples

In [4]:
def bgr_to_rgb(bgr):
    return cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

def overlay_mask_on_bgr(bgr, mask, color=(0,0,255), alpha=0.35):
    overlay = bgr.copy()
    c = np.zeros_like(bgr)
    c[:,:,:] = np.array(color, dtype=np.uint8)
    overlay[mask==1] = (alpha*c[mask==1] + (1-alpha)*overlay[mask==1]).astype(np.uint8)
    return overlay

def mask_to_3ch(mask01):
    return (mask01 * 255).astype(np.uint8)

sample_n = 12
sample_df = df.sample(min(sample_n, len(df)), random_state=RANDOM_STATE)

saved = 0
skipped_no_annot = 0

for i, r in enumerate(sample_df.to_dict("records"), start=1):
    img_path = r["path"]
    bgr = cv2.imread(img_path)
    if bgr is None:
        continue

    txt = find_annotation_txt(img_path)
    if txt is None:
        skipped_no_annot += 1
        continue

    labels, lesion_mask, roi_mask, score = segment_guided(bgr, txt)
    if roi_mask is None:
        continue

    label_vis = (labels.astype(np.float32) / max(1, labels.max()) * 255).astype(np.uint8)
    label_vis = cv2.applyColorMap(label_vis, cv2.COLORMAP_JET)

    over_lesion = overlay_mask_on_bgr(bgr, lesion_mask, color=(0,255,255), alpha=0.35)  # yellow-ish
    over_roi    = overlay_mask_on_bgr(bgr, roi_mask,    color=(0,0,255),   alpha=0.35)  # red

    fig = plt.figure(figsize=(18, 4))
    ax1 = fig.add_subplot(1, 4, 1)
    ax2 = fig.add_subplot(1, 4, 2)
    ax3 = fig.add_subplot(1, 4, 3)
    ax4 = fig.add_subplot(1, 4, 4)

    ax1.imshow(bgr_to_rgb(bgr))
    ax1.set_title(f"Original - {r['class']}")
    ax1.axis("off")

    ax2.imshow(bgr_to_rgb(label_vis))
    ax2.set_title("KMeans (k=3) clusters (LAB)")
    ax2.axis("off")

    ax3.imshow(bgr_to_rgb(over_lesion))
    ax3.set_title("Annotation lesion mask (YOLO)")
    ax3.axis("off")

    ax4.imshow(bgr_to_rgb(over_roi))
    ax4.set_title(f"Chosen ROI cluster (overlap={score:.2f})")
    ax4.axis("off")

    out_img = SAMPLE_DIR / f"seg_{saved+1:02d}_{r['class']}.png"
    fig.savefig(out_img, bbox_inches="tight")
    plt.close(fig)
    saved += 1

print("Saved samples:", saved)
print("Skipped (no annotation found):", skipped_no_annot)
print("Output folder:", SAMPLE_DIR)


c:\Users\User\.conda\envs\pengenalan-pola\Lib\site-packages\threadpoolctl.py:1226: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


Saved samples: 7
Skipped (no annotation found): 5
Output folder: E:\Kuliah\Pengenalan Pola\final-project\outputs\samples\segmentation_guided
